In [ ]:
!pip install langchain
!pip install openai
!pip install google-search-results

In [2]:
serpapi_key = 'SERPAPI_KEY'
open_api_key = 'OPEN_API_KEY'

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain 
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper 
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import load_tools
from langchain.agents import AgentTypeª



In [3]:
# define the llm model

llm = OpenAI(openai_api_key=open_api_key, temperature=0.8) 

NameError: name 'open_api_key' is not defined

In [5]:
# Define your job-to-be-done and customer to be explored

customer_description = 'a family consisting of two adults and two children with average income who are renting anu apartment in a big city'
job_to_be_done = 'monitoring and paying energy bills'

In [6]:
# Prompt hardest part of job to be done for customer
hardest_template = PromptTemplate(
    input_variables = ['customer_description','job_to_be_done'], 
    template='You are {customer_description}. What is the hardest part about {job_to_be_done}?')

hardest_chain = LLMChain(llm=llm, prompt=hardest_template, verbose=True, output_key='hardest_part')

hardest = hardest_chain.run(customer_description=customer_description, job_to_be_done=job_to_be_done)

print(hardest)



> Entering new LLMChain chain...
Prompt after formatting:
You are a family consisting of two adults and two children with average income who are renting anu apartment in a big city. What is the hardest part about monitoring and paying energy bills?

> Finished chain.


The hardest part about monitoring and paying energy bills for a family of two adults and two children renting an apartment in a big city is trying to keep costs under control. Big city living is expensive, and energy costs can add up quickly. It is important to be aware of energy usage and find ways to conserve, such as using LED light bulbs, unplugging appliances when not in use, and using the most efficient appliances available. Additionally, it can be difficult to stay up to date with energy bills and make sure that payments are made on time to avoid late fees and penalties.


In [7]:
# Prompt value proposition to solve hardest part of job-to-be-done

value_proposition_template = PromptTemplate(
    input_variables = ['hardest_part'], 
    template='Create a unique value proposition for a startup that solves the problem of {hardest_part}')

value_proposition_chain = LLMChain(llm=llm, prompt=value_proposition_template, verbose=True, output_key='value_proposition')

valueproposition = value_proposition_chain.run(hardest_part=hardest)

print(valueproposition)




> Entering new LLMChain chain...
Prompt after formatting:
Create a unique value proposition for a startup that solves the problem of 

The hardest part about monitoring and paying energy bills for a family of two adults and two children renting an apartment in a big city is trying to keep costs under control. Big city living is expensive, and energy costs can add up quickly. It is important to be aware of energy usage and find ways to conserve, such as using LED light bulbs, unplugging appliances when not in use, and using the most efficient appliances available. Additionally, it can be difficult to stay up to date with energy bills and make sure that payments are made on time to avoid late fees and penalties.

> Finished chain.


Our startup offers a unique value proposition for families in a big city who are trying to manage their energy costs. Our platform will provide real-time monitoring and insights into energy usage, helping you to understand what you’re using and how you can 

In [8]:
# Prompt description value proposition canvas

canvas_template = PromptTemplate(
    input_variables = ['value_proposition'], 
    template='Provide the value proposition canvas for {value_proposition}. You should describe the following elements: customer jobs, custome pains, customer gains, products and services, pain relievers, gain creators')

canvas_chain = LLMChain(llm=llm, prompt=canvas_template, verbose=True, output_key='canvas')

canvas = canvas_chain.run(value_proposition=valueproposition)

print(canvas)





> Entering new LLMChain chain...
Prompt after formatting:
Provide the value proposition canvas for 

Our startup offers a unique value proposition for families in a big city who are trying to manage their energy costs. Our platform will provide real-time monitoring and insights into energy usage, helping you to understand what you’re using and how you can save money. We’ll also make it easy to pay your energy bills on time, with automatic payments and reminders to help you stay on top of bills. Our platform is easy to use, and with our helpful tools and resources, you’ll be able to manage your energy costs with confidence.. You should describe the following elements: customer jobs, custome pains, customer gains, products and services, pain relievers, gain creators

> Finished chain.
, key activities, channels, customer relationships, revenue streams and cost structure. 

Customer Jobs: Families in a big city want to manage their energy costs without taking too much time or effort. 



In [12]:
# Prompt identification of core competitors for this value proposition

competitors_template = PromptTemplate(
    input_variables = ['value_proposition'], 
    template='Find three existing startup companies that have a description that is very similar to the following description: {value_proposition}. For each startup, provide company name, location and website.')

competitors_chain = LLMChain(llm=llm, prompt=competitors_template, verbose=True, output_key='competitors')

competitors = competitors_chain.run(value_proposition=valueproposition)

print(competitors)



> Entering new LLMChain chain...
Prompt after formatting:
Find three existing startup companies that have a description that is very similar to the following description: 

Our startup offers a unique value proposition for families in a big city who are trying to manage their energy costs. Our platform will provide real-time monitoring and insights into energy usage, helping you to understand what you’re using and how you can save money. We’ll also make it easy to pay your energy bills on time, with automatic payments and reminders to help you stay on top of bills. Our platform is easy to use, and with our helpful tools and resources, you’ll be able to manage your energy costs with confidence.. For each startup, provide company name, location and website.

> Finished chain.


1. WattTime, Oakland, CA, https://watttime.org/
2. ohmconnect, San Francisco, CA, https://www.ohmconnect.com/
3. EnergyHub, New York, NY, https://energyhub.com/


In [18]:
# Prompt collecting additional information on competitors via SerpAPI


competitorsselect = competitors[5:]

# split string based on .

competitors_list = competitorsselect.split('\n')

competitors_list

# get informatin for competitors

search = SerpAPIWrapper(serpapi_api_key = serpapi_key)
search_tool = [Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search")]

self_ask_with_search = initialize_agent(search_tool, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)

descriptionlist = []

for i in range(len(competitors_list)):
    company = competitors_list[i]
    searchstring = "Provide company information for " + company + ". Include company description and founders"
    result = self_ask_with_search.run(searchstring)
    descriptionlist.append(result)

print(descriptionlist)



> Entering new AgentExecutor chain...
 Yes.
Follow up: What is WattTime?
Intermediate answer: WattTime is an environmental tech nonprofit that empowers all people, companies, policymakers, and countries to slash emissions and choose cleaner energy.

Follow up: Who are the founders of WattTime?
Intermediate answer: Gavin McCormick is co-founder and executive director of WattTime. Academic and social entrepreneur, Gavin applies behavioral economics to energy markets to develop ultra-low-cost techniques to deliver more renewable power and energy efficiency at scale.

Danielle Fugere is co-founder and president of WattTime. She has a background in clean energy policy and political action, having most recently served as the Climate Law Institute's President and Chief Counsel. 

Dr. Alexander “Sandy” MacDonald is the Chief Science Officer at WattTime, and has over 25 years of experience in energy system analytics, optimization, and control. 

So the final answer is: WattTime is an environm

In [14]:
# Prompt Generation of hypothese for testing value proposition

hypotheses_template = PromptTemplate(
    input_variables = ['value_proposition', 'canvas'], 
    template='Generate a list of hypotheses for testing the following value proposition: {value_proposition}. You should rely on information from the value proposition canvas: {canvas}. Include at least 3 hypotheses. Each hypothesis should focus on the feasibility or viability of the business model. Here are some potential topics for hypotheses testing: willingness to pay, preferred distribution model, ultimate customer segmet, etc.')

hypotheses_chain = LLMChain(llm=llm, prompt=hypotheses_template, verbose=True, output_key='hypotheses')

hypotheses = hypotheses_chain.run(value_proposition=valueproposition, canvas=canvas)

print(hypotheses)



> Entering new LLMChain chain...
Prompt after formatting:
Generate a list of hypotheses for testing the following value proposition: 

Our startup offers a unique value proposition for families in a big city who are trying to manage their energy costs. Our platform will provide real-time monitoring and insights into energy usage, helping you to understand what you’re using and how you can save money. We’ll also make it easy to pay your energy bills on time, with automatic payments and reminders to help you stay on top of bills. Our platform is easy to use, and with our helpful tools and resources, you’ll be able to manage your energy costs with confidence.. You should rely on information from the value proposition canvas: , key activities, channels, customer relationships, revenue streams and cost structure. 

Customer Jobs: Families in a big city want to manage their energy costs without taking too much time or effort. 

Customer Pains: Difficulty understanding energy usage and when

In [20]:
# Write input and output to text file

with open('lean startup description.txt', 'w') as f:
    f.write('Input job to be done = ' + job_to_be_done + '\n')
    f.write('Input customer description = ' + customer_description + '\n')
    f.write('\n' + 'GPT output hardest part of job to be done = ' + hardest +  '\n')
    f.write('\n' + 'GPT output value proposition = ' + valueproposition +  '\n')
    f.write('\n' + 'GPT output value proposition canvas = ' + canvas +  '\n')
    f.write('\n' + 'GPT output most similar competitors = ' + competitors +  '\n')
    f.write('\n' + 'GPT output company description competitor 1 = ' + str(descriptionlist[0]) +  '\n')
    f.write('\n' + 'GPT output company description competitor 2 = ' + str(descriptionlist[1]) +  '\n')
    f.write('\n' + 'GPT output company description competitor 3 = ' + str(descriptionlist[2]) +  '\n')
    f.write('\n' + 'GPT output hypotheses = ' + hypotheses +  '\n')